In [45]:
# Load packages
#import tensorflow as tf
#from tensorflow import keras
import numpy as np
import pandas as pd
import os
import scipy as scp
import scipy.stats as scps
from scipy.optimize import differential_evolution
from scipy.optimize import minimize
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Load my own functions
#import dnnregressor_train_eval_keras as dnnk
#import make_data_wfpt as mdw
#from kde_training_utilities import kde_load_data
import ddm_data_simulation as ddm_sim

In [46]:
dir_ = '/media/data_cifs/afengler/data/michael_bg_stn/'
file_list = os.listdir(dir_)

# potentially delete data files
# here

In [48]:
file_list

['loconf7_go_stnhi.txt',
 'loconf7_go_stnlo.txt',
 'hiconf_go_stnmid.txt',
 'loconf7_go_stnmid.txt',
 'bg_stn_full.csv',
 'hiconf_go_stnhi.txt',
 'bg_stn_sampling_ready.pickle',
 'hiconf_go_stnlo.txt']

In [47]:
pd_list = []

for file_ in file_list:
    pd_tmp = pd.read_csv(dir_ + file_, sep = '\t')
    pd_tmp['file'] = file_
    pd_list.append(pd_tmp)

data = pd.concat(pd_list)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [49]:
pd_tmp = pd.read_csv(dir_ + 'bg_stn_full.csv')
data = pd_tmp

In [11]:
# Preprocess
data['rt'] = (data['avg_cycles']*10) / 1000
data.loc[data['cnt_err'] == 0, 'cnt_err'] = -1
data['choice'] = data['cnt_err']

In [ ]:
# Write to file
data.to_csv(dir_ + 'bg_stn_full.csv', index = False)

In [95]:
# Read from file if desired
data = pd.read_csv(dir_ + 'bg_stn_full.csv')

In [ ]:
data.iloc[:, -2:]

In [96]:
files_ = np.unique(data['file'])

In [ ]:
list(files_)

In [97]:
data_arr = np.zeros((len(files_), int(data.shape[0] / len(files_)), 2))

In [98]:
cnt = 0
step = 5000
for file_ in files_:
    data_arr[cnt] = data[data['file'] == file_].iloc[:, -2:]
    cnt += 1

In [54]:
import pickle 
pickle.dump((data_arr, list(files_)), open('/media/data_cifs/afengler/bg_stn_sampling_ready.pickle', 'wb'))

In [53]:
files_

array(['hiconf_go_stnhi.txt', 'hiconf_go_stnlo.txt',
       'hiconf_go_stnmid.txt', 'loconf7_go_stnhi.txt',
       'loconf7_go_stnlo.txt', 'loconf7_go_stnmid.txt'], dtype=object)

In [99]:
data_arr

array([[[ 0.53,  1.  ],
        [ 0.69, -1.  ],
        [ 0.48,  1.  ],
        ...,
        [ 0.56, -1.  ],
        [ 0.58,  1.  ],
        [ 0.72, -1.  ]],

       [[ 0.51,  1.  ],
        [ 0.48,  1.  ],
        [ 0.47,  1.  ],
        ...,
        [ 0.49,  1.  ],
        [ 0.52,  1.  ],
        [ 0.59,  1.  ]],

       [[ 0.52,  1.  ],
        [ 0.67, -1.  ],
        [ 0.45, -1.  ],
        ...,
        [ 0.5 ,  1.  ],
        [ 0.38, -1.  ],
        [ 0.52, -1.  ]],

       [[ 0.52,  1.  ],
        [ 0.68,  1.  ],
        [ 0.48,  1.  ],
        ...,
        [ 0.59,  1.  ],
        [ 0.55, -1.  ],
        [ 0.65,  1.  ]],

       [[ 0.5 ,  1.  ],
        [ 0.45,  1.  ],
        [ 0.42,  1.  ],
        ...,
        [ 0.49,  1.  ],
        [ 0.42,  1.  ],
        [ 0.6 ,  1.  ]],

       [[ 0.51,  1.  ],
        [ 0.47,  1.  ],
        [ 0.5 ,  1.  ],
        ...,
        [ 0.81, -1.  ],
        [ 0.57,  1.  ],
        [ 0.54,  1.  ]]])

In [59]:
pickle.load(open('/media/data_cifs/afengler/data/real/bg_stn_sampling_ready.pickle', 'rb'))

(array([[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 1.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]]),
 ['hiconf_go_stnhi.txt',
  'hiconf_go_stnlo.txt',
  'hiconf_go_stnmid.txt',
  'loconf7_go_stnhi.txt',
  'loconf7_go_stnlo.txt',
  'loconf7_go_stnmid.txt'])

In [61]:
data_arr[1]

array([[0.51, 1.  ],
       [0.48, 1.  ],
       [0.47, 1.  ],
       ...,
       [0.49, 1.  ],
       [0.52, 1.  ],
       [0.59, 1.  ]])

In [ ]:
for i in zip(data_arr,['random' for i in range(data_arr.shape[0])]):
    print(i)

In [80]:
data_arr

array([[[530.,   1.],
        [690.,  -1.],
        [480.,   1.],
        ...,
        [560.,  -1.],
        [580.,   1.],
        [720.,  -1.]],

       [[ 51.,   1.],
        [ 48.,   1.],
        [ 47.,   1.],
        ...,
        [ 49.,   1.],
        [ 52.,   1.],
        [ 59.,   1.]],

       [[ 52.,   1.],
        [ 67.,  -1.],
        [ 45.,  -1.],
        ...,
        [ 50.,   1.],
        [ 38.,  -1.],
        [ 52.,  -1.]],

       [[ 52.,   1.],
        [ 68.,   1.],
        [ 48.,   1.],
        ...,
        [ 59.,   1.],
        [ 55.,  -1.],
        [ 65.,   1.]],

       [[ 50.,   1.],
        [ 45.,   1.],
        [ 42.,   1.],
        ...,
        [ 49.,   1.],
        [ 42.,   1.],
        [ 60.,   1.]],

       [[ 51.,   1.],
        [ 47.,   1.],
        [ 50.,   1.],
        ...,
        [ 81.,  -1.],
        [ 57.,   1.],
        [ 54.,   1.]]])

In [9]:
pd_tmp

,batch,epoch,avg_sse,cnt_err,avg_cycles,file,rt,choice
0,0,0,1.318510,1,52,loconf7_go_stnhi.txt,0.52,1
1,0,1,1.018420,1,68,loconf7_go_stnhi.txt,0.68,1
2,0,2,1.372920,1,48,loconf7_go_stnhi.txt,0.48,1
3,0,3,1.451170,1,56,loconf7_go_stnhi.txt,0.56,1
4,0,4,1.024080,1,52,loconf7_go_stnhi.txt,0.52,1
5,0,5,1.731510,1,40,loconf7_go_stnhi.txt,0.40,1
6,0,6,1.563470,1,46,loconf7_go_stnhi.txt,0.46,1
7,0,7,1.181070,1,61,loconf7_go_stnhi.txt,0.61,1
8,0,8,1.430380,1,45,loconf7_go_stnhi.txt,0.45,1
9,0,9,1.516130,1,64,loconf7_go_stnhi.txt,0.64,1


In [115]:
import pickle 
data = pickle.load(open('/media/data_cifs/afengler/data/real/bg_stn_sampling_ready.pickle', 'rb'))

In [91]:
import dataset_generator as dg
data

(array([[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 1.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],
 
        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]]),
 ['hiconf_go_stnhi.txt',
  'hiconf_go_stnlo.txt',
  'hiconf_go_stnmid.txt',
  'loconf7_go_stnhi.txt',
  'loconf7_go_stnlo.txt',
  'loconf7_go_stnmid.txt'])

In [72]:
data_tmp

array([[ 5.1,  1. ],
       [ 4.7,  1. ],
       [ 5. ,  1. ],
       ...,
       [ 8.1, -1. ],
       [ 5.7,  1. ],
       [ 5.4,  1. ]])

In [29]:
bin_arbitrary_fptd(out = data[0][0])

array([[  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  6.,  33.],
       [134., 265.],
       [491., 896.],
       [500., 862.],
       [312., 485.],
       [161., 244.],
       [ 87., 130.],
       [ 60., 112.],
       [ 51.,  74.],
       [ 26.,  30.],
       [ 15.,  17.],
       [  2.,   4.],
       [  0.,   2.],
       [  0.,   1.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,

In [94]:
data[0]

KeyError: 0

In [110]:
binned_data = np.zeros((data_arr.shape[0], 256, 2))
for i in range(binned_data.shape[0]):
    data_tmp = data_arr[i].copy()
    data_tmp[:, 0] = data_tmp[:, 0] * 10
    binned_data[i] = bin_arbitrary_fptd(out = data_tmp)

In [116]:
pickle.dump((binned_data, data[1]), 
            open('/media/data_cifs/afengler/data/real/bg_stn_binned_scaled.pickle', 'wb'), protocol = 2)

In [104]:
data_arr.shape[0]

6

In [114]:
data

,batch,epoch,avg_sse,cnt_err,avg_cycles,file,rt,choice
0,0,0,1.318510,1,52,loconf7_go_stnhi.txt,0.52,1
1,0,1,1.018420,1,68,loconf7_go_stnhi.txt,0.68,1
2,0,2,1.372920,1,48,loconf7_go_stnhi.txt,0.48,1
3,0,3,1.451170,1,56,loconf7_go_stnhi.txt,0.56,1
4,0,4,1.024080,1,52,loconf7_go_stnhi.txt,0.52,1
5,0,5,1.731510,1,40,loconf7_go_stnhi.txt,0.40,1
6,0,6,1.563470,1,46,loconf7_go_stnhi.txt,0.46,1
7,0,7,1.181070,1,61,loconf7_go_stnhi.txt,0.61,1
8,0,8,1.430380,1,45,loconf7_go_stnhi.txt,0.45,1
9,0,9,1.516130,1,64,loconf7_go_stnhi.txt,0.64,1


In [112]:
binned_data[0]

array([[  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  0.,   0.],
       [  1.,   0.],
       [  0.,   0.],
       [  0.,   3.],
       [  0.,   1.],
       [  0.,   1.],
       [  0.,   0.],
       [  1.,   4.],
       [  0.,

In [117]:
import scipy 
scipy.random.seed()
